In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import json
import pandas as pd
from pathlib import Path
from copy import deepcopy
from functools import partial

from bellek.utils import set_seed, jprint
from bellek.musique.eval import calculate_metrics, compare_answers

set_seed(89)

In [3]:
from tqdm.auto import tqdm
tqdm.pandas()

In [6]:
def report_scores(pred_path: Path):

    df = pd.read_json('../../data/generated/musique-evaluation/dataset.jsonl', orient='records', lines=True).iloc[100:200]
    baseline_df = pd.read_json(pred_path, orient='records', lines=True)
    df = pd.merge(df, baseline_df.drop(columns=['predicted_support_idxs', 'predicted_answerable']), on='id', suffixes=('', ''))
    df = compare_answers(df)
    scores = calculate_metrics(df)
    print("-"*80)
    print(pred_path.stem.replace('musique_ans_v1.0_', ''))
    jprint(scores)

In [7]:
BASELINE_PREDS_DIR = Path('/Users/bdsaglam/dev/repos/bellek/tmp/predictions/')

for pred_path in BASELINE_PREDS_DIR.glob('musique_ans_v1.0_*.jsonl'):
    report_scores(pred_path)

--------------------------------------------------------------------------------
dev_select_answer_model_predictions
{
  "exact_match": 0.44,
  "f1": 0.5455868388406016
}
--------------------------------------------------------------------------------
dev_step_execution_by_select_answer_model_predictions
{
  "exact_match": 0.64,
  "f1": 0.6975436720142603
}
--------------------------------------------------------------------------------
dev_end2end_model_predictions
{
  "exact_match": 0.36,
  "f1": 0.44844027653699586
}
--------------------------------------------------------------------------------
dev_step_execution_by_end2end_model_predictions
{
  "exact_match": 0.62,
  "f1": 0.6934616127978412
}
